#Assignment 5
Group members: Sebastian Vinther

#Installs
loading libraries

In [1]:
!pip install --upgrade pip
!pip install gradio==3.50.2
!pip install accelerate --q
!pip install "pydantic>=1.9,<2.0"
!pip install pypdf --q
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.18.0 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.16.4 which is incompatible.
peft 0.10.0 requires huggingface-hub>=0.17.0, but you have huggingface-hub 0.16.4 which is incompatible.
transformers 4.39.3 requires huggingface-hub<1.0,>=0.19.3, but you have huggingface-hub 0.16.4 which is incompatible.


#Importing Libraries

In [8]:
!pip install --upgrade transformers

In [3]:
# Cloning Github repository to Google Files
!git clone https://github.com/sebvinther/assignment5

fatal: destination path 'assignment5' already exists and is not an empty directory.


In [4]:
from langchain.document_loaders import PyPDFDirectoryLoader
# Loading the PDF files from Google Files
loader = PyPDFDirectoryLoader("/content/assignment5/machinelearning.pdf")
docs = loader.load()
# Printing the number of pages of the loaded data
len(docs)

0

In [21]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached torch-2.2.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 6.8 MB/s eta 0:00:00
Using cached torch-2.2.1-cp310-cp310-manylinux1_x86_64.whl (755.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.21 requires torch==2.0.1, but you have torch 2.2.1 which is incompatible.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import PromptTemplate

# Corrected imports
from langchain.document_loaders import UnstructuredMarkdownLoader, PyPDFLoader
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline

from textwrap import fill
import torch

import requests

In [5]:
!locale  # Use in Jupyter/Colab notebook

LANG=en_US.UTF-8
LANGUAGE=en_US
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=en_US.UTF-8


#Set-up

Loading our document directly from github repository

In [6]:
pdf_url = "https://github.com/sebvinther/assignment5/raw/main/machinelearning.pdf"
response = requests.get(pdf_url)

with open("file.pdf", "wb") as file:
    file.write(response.content)

print("PDF file downloaded successfully.")


PDF file downloaded successfully.


In [7]:
loader = PyPDFLoader("/content/file.pdf")

docs = loader.load()
len(docs)

234

Dividing the document into more manageable segments for quicker processing. Each segment is 1024 characters in length, with a 64-character overlap between segments to more effectively maintain context.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

513

This code initializes a text embedding model using Hugging Face Transformers. It sets parameters for model usage and embedding processing, then embeds the text content of the first document chunk.

In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

1024


The following code leverages the Chroma library to establish a database (db) that archives embeddings derived from supplied texts. It designates a directory to store the database persistently. Subsequently, it executes a similarity query within this database for documents that resemble the query "Advantages", fetching the foremost 2 matches.

In [11]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("What is machine learning?", k=2)
print(results[0].page_content)

INTRODUCTION TO MACHINE LEARNING



This code establishes a pipeline for generating text by utilizing a pre-trained language model, tokenizer, and a set of generation parameters. It configures specific options to regulate attributes such as randomness and variety in the text produced.

In [12]:
MODEL_NAME = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

#Creating Question answer chain and trying different prompt approaches

##Template 1 and first queries

In [13]:
template1 = """
<s>[INST] <<SYS>>
As a Data Scientist, your role involves dissecting the forthcoming data to extract meaningful insights addressing the final query. Aim for clarity and precision in your explanation, ensuring it remains accessible without delving too deep into complexity
<</SYS>>

{context}

{question} [/INST]
"""

prompt1 = PromptTemplate(template=template1, input_variables=["context", "question"])


qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt1},
)

In [14]:
result1 = qa_chain1("What is Machine learning?")

print(fill(result1["result"].strip(),width=80))

Great! Let's dive into the world of machine learning. As a Data Scientist, I'll
explain what machine learning is, its types, applications, and why it's
important in today's technology landscape. Machine learning is a subfield of
artificial intelligence that involves training algorithms to learn from data and
make predictions or decisions based on that data. In simple terms, machine
learning is a way for computers to learn from experience and improve their
performance over time. There are several types of machine learning, including:
1. Supervised learning: In this type of machine learning, the algorithm is
trained on labeled data, where the correct output is already known. The
algorithm learns to map inputs to outputs based on the labeled examples. Once
the algorithm is trained, it can make predictions on new, unseen data. Examples
of supervised learning include image classification, sentiment analysis, and
spam detection. 2. Unsupervised learning: In this type of machine learning, the

In [15]:
result2 = qa_chain1("What advantages does machine learning have over traditional statistics?")

print(fill(result2["result"].strip(),width=80))

As a Data Scientist, I must say that machine learning offers several advantages
over traditional statistics. Here are some key benefits: 1. Predictive Power:
Machine learning algorithms can analyze large datasets and identify patterns
that may not be apparent through traditional statistical methods. This allows us
to make more accurate predictions about future events or trends. 2. Automated
Processing: Machine learning algorithms can process vast amounts of data much
faster than humans, making them ideal for analyzing big data sets. This saves
time and resources, allowing us to gain insights quickly and efficiently. 3.
Adaptability: Machine learning algorithms can adapt to changing data
distributions over time, enabling them to maintain accuracy even when the
underlying data changes. Traditional statistical methods often struggle with
these types of changes, leading to reduced accuracy. 4. Handling Uncertainty:
Machine learning algorithms can handle uncertainty by providing probability

In [20]:
result3 = qa_chain1("What are some challenges or limitations of machine learning?")

print(fill(result3["result"].strip(),width=80))

As a Data Scientist, I must first emphasize that machine learning is a powerful
tool for extracting insights from large datasets. However, like any other
technique, it has its share of challenges and limitations. Here are some of the
most common ones: 1. **Data Quality Issues**: Machine learning algorithms
require high-quality data to produce accurate results. If the data is noisy,
incomplete, or biased, the models may not perform well or make unfair
assumptions. Therefore, it's crucial to ensure that the data is clean,
consistent, and representative of the problem you're trying to solve. 2.
**Overfitting**: Overfitting occurs when a model is too complex and learns the
noise in the training data rather than the underlying patterns. This can result
in poor generalization performance on unseen data. To avoid overfitting,
regularization techniques such as L1 and L2 regularization can be applied. 3.
**Understanding the Model**: As machines learn, it can be difficult to interpret
the decisi

##Template 2 and second queries

In [21]:
template2 = """
<s>[INST] <<SYS>>
As a Data Scientist, you are assigned to sift through the provided information and deliver insights pertinent to the question at the conclusion. Your explanation should be both succinct and accurate, while remaining accessible by not delving into overly complex terminology., if possible use bullet points
<</SYS>>

{context}

{question} [/INST]
"""

prompt2 = PromptTemplate(template=template2, input_variables=["context", "question"])


qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt2},
)

In [22]:
result4 = qa_chain2("What is machine learning?")

print(fill(result4["result"].strip(),width=80))

Great! I'd be happy to help you understand what machine learning is. Here are
some key points to keep in mind: * Machine learning is a subfield of artificial
intelligence that involves training algorithms to learn from data and make
predictions or decisions based on that data. * The goal of machine learning is
to enable machines to learn from experience and improve their performance on a
task without being explicitly programmed for that task. * Machine learning
algorithms can be divided into two main categories: supervised and unsupervised
learning. Supervised learning involves training an algorithm on labeled data,
where the correct output is already known, and the algorithm learns to predict
the correct output for new, unseen data. Unsupervised learning involves training
an algorithm on unlabeled data, and the algorithm learns patterns and
relationships in the data without any prior knowledge of the correct output. *
Some common applications of machine learning include image and spee

In [23]:
result5 = qa_chain2("What advantages does machine learning have over traditional statistics?")

print(fill(result5["result"].strip(),width=80))

As a data scientist, I can provide you with some key benefits of machine
learning compared to traditional statistics: 1. **Scalability**: Machine
learning algorithms can handle large datasets and perform well even when dealing
with massive amounts of data. Traditional statistical methods may struggle with
such volumes of data, making machine learning more practical for big data
analysis. 2. **Flexibility**: Machine learning models can adapt to various types
of data and can learn patterns that would be difficult or impossible to detect
using traditional statistical techniques. For example, machine learning can
identify patterns in image recognition tasks, natural language processing, or
predicting user behavior based on past data. 3. **Automation**: Machine learning
automates many aspects of data analysis, including feature engineering, model
selection, and hyperparameter tuning. This automation saves time and reduces the
risk of human error, allowing data analysts to focus on higher-le

In [24]:
result6 = qa_chain2("What are some challenges or limitations of machine learning?")

print(fill(result6["result"].strip(),width=80))

As a data scientist, I can provide you with some common challenges and
limitations of machine learning that you may encounter: Challenges: 1. **Data
Quality**: Machine learning algorithms require high-quality data to produce
accurate results. Poor data quality can lead to biased models, reduced accuracy,
and decreased performance. Ensuring data cleanliness, completeness, and
consistency is crucial before training machine learning models. 2.
**Overfitting**: Overfitting occurs when a model becomes too complex and starts
to fit the noise in the training data rather than the underlying patterns. This
can result in poor generalization performance on unseen data. Techniques like
regularization, early stopping, and cross-validation can help mitigate
overfitting. 3. **Model Interpretability**: Machine learning models can be
difficult to interpret, making it challenging to understand why they make
certain predictions. This lack of transparency can be problematic in
applications where explainab

##Template 3 and third queries

In [25]:
template3 = """
<s>[INST] <<SYS>>
As a deep learning specialist, your role involves scrutinizing the upcoming data within this context to derive meaningful insights for the posed question. Start with a brief introduction to frame your analysis.
Proceed to explain your findings clearly and precisely, aiming your explanation at data science students to enhance their understanding.
Conclude by succinctly addressing the question, tying together the insights you've gathered from the analysis.
<</SYS>>

{context}

{question} [/INST]
"""

prompt3 = PromptTemplate(template=template3, input_variables=["context", "question"])


qa_chain3 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt3},
)

In [27]:
result7 = qa_chain3("What is machine learning?")

print(fill(result7["result"].strip(),width=80))

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    a

Introduction: Machine learning is a subfield of artificial intelligence that
involves training algorithms to learn from data and make predictions or
decisions based on that data. In other words, machine learning is a way for
computers to learn from experience and improve their performance on a task
without being explicitly programmed for that task. The goal of machine learning
is to enable computers to perform tasks that typically require human
intelligence, such as visual perception, speech recognition, and language
translation. In this analysis, I will be focusing on the concept of machine
learning and its applications in various industries. I will also provide some
examples of real-world use cases where machine learning has been successfully
implemented. Findings: Machine learning is a rapidly growing field, with
numerous applications across various industries. Some of the key applications of
machine learning include: 1. Image Recognition: Machine learning algorithms can
be trained 

In [29]:
result8 = qa_chain2("What advantages does machine learning have over traditional statistics?")

print(fill(result8["result"].strip(),width=80))

As a data scientist, I can provide you with some key benefits of machine
learning compared to traditional statistics: 1. **Scalability**: Machine
learning algorithms can handle large datasets and perform well even when dealing
with massive amounts of data. Traditional statistical methods may struggle with
such volumes of data, making machine learning more practical for big data
analysis. 2. **Flexibility**: Machine learning models can adapt to various types
of data and can learn patterns that would be difficult or impossible to detect
using traditional statistical techniques. For example, machine learning can
identify patterns in image recognition tasks, natural language processing, or
predicting user behavior based on past data. 3. **Automation**: Machine learning
automates many aspects of data analysis, including feature engineering, model
selection, and hyperparameter tuning. This automation saves time and reduces the
risk of human error, allowing data analysts to focus on higher-le

In [30]:
result9 = qa_chain2("What are some challenges or limitations of Machine learning?")

print(fill(result9["result"].strip(),width=80))

As a data scientist, I can provide you with some common challenges and
limitations of machine learning that you may encounter: Challenges: 1. **Data
Quality**: The quality of the data used for training machine learning models is
crucial. Poor quality data can lead to biased or inaccurate models. Ensuring
that the data is clean, complete, and representative of the problem domain is
essential. 2. **Overfitting**: Overfitting occurs when a model is too complex
and learns the noise in the training data rather than the underlying patterns.
This can result in poor generalization performance on unseen data. Techniques
such as regularization, early stopping, and cross-validation can help mitigate
overfitting. 3. **Understanding Model Behavior**: As machine learning models
become more complex, it can be difficult to understand why they make certain
predictions. Lack of interpretability can limit our ability to trust the models
and understand their decision-making process. 4. **Handling Missing 


---


## the 3rd prompt, which outputs the most detailed answer based on the same queries. This is of course at the cost of the time it takes for the query to process ---- the app is just the demonstration.

#Gradio application

In [33]:
import gradio as gr
def rag_qa(query):
    result1 = qa_chain1(query)  # Use query as both context and question
    print("Result from rag_qa function:", result1)  # Print the result
    return result1["result"].strip()

# Create Gradio interface
interface = gr.Interface(fn=rag_qa, inputs="textbox", outputs="text", title="RAG Information Retrieval - Machine Learning", description="Ask ME! about Machine Learning")
interface.queue().launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c50be1c4c622501014.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
